In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import math

width = 505.89

inputnum_arr = np.array([1,2,3,4])
blknum_arr = np.array([10,20,50,100,200,500,1000])
datasz_arr = np.array([8192,65536,524288,4194304])
datasz_labels = np.array(["8KB", "64KB", "512KB", "4MB"])

numctxt_arr = np.array([10,20,50,100,200,500,1000])
numpart_arr = np.array([10,20,50,100,200,500,1000])

In [ ]:
def set_size(width, fraction=1):
    """ Set aesthetic figure dimensions to avoid scaling in latex.
    Parameters
    ----------
    width: float
            Width in pts
    fraction: float
            Fraction of the width which you wish the figure to occupy
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [ ]:
def plot_kv(input_path, output_path):
    # VERIFY_KV (1 plot, bar graph, # of inputs on x-axis)
    w = 0.1
    maxw = w*3
    colors = ['darksalmon','burlywood','darkseagreen','cornflowerblue','thistle','rosybrown','silver']

    fig, ax = plt.subplots(1,1,figsize=set_size(width, fraction=1))

    input_data = pd.read_csv(input_path)
    x = np.arange(len(inputnum_arr))
    y_values = list()

    # Get latency values by block_num. lists in "values" is what we want to plot
    for blk_num in blknum_arr:
        y_values.append(input_data.loc[input_data['block_num'] == blk_num]['total'].tolist())
       
    for i in range(len(y_values)):
        ax.bar(x+(-maxw+i*w), y_values[i], w, color=colors[i])

    plt.xticks(x,inputnum_arr, fontsize=12)
    
    glob_max = max(y_values[-1])
    step_size = 0.5
    new_yts = np.arange(0, glob_max+step_size, step_size)
    plt.yticks(new_yts, new_yts, fontsize=12)    
#     plt.yticks(fontsize=12)

    plt.xlabel("Number of dependencies", fontsize=12)
    plt.ylabel("Latency (s)", fontsize=12)
    
    plt.legend(blknum_arr.astype(str), title_fontsize=12, fontsize=12, title="Number of blocks", ncol=1, bbox_to_anchor=(1,0.8))
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.5)
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
v_kv_path = "./data/kv.csv"
plot_kv(v_kv_path, "./plots/kv.pdf")

In [ ]:
def plot_kv_subplot(input_path, output_path, isLocal, isLog):
    fig, axs = plt.subplots(2,2,figsize=set_size(width*1.2, fraction=1))
    colors = ['coral','mediumseagreen', 'dodgerblue', 'darkorchid']

    input_data = pd.read_csv(input_path)
    y_values = list()

    for input_num in inputnum_arr:
        y_values.append(input_data.loc[input_data['input_num'] == input_num]['total'].tolist())

    print(y_values)
        
    if isLog:
        x = blknum_arr
        bar_w = 0.2 * np.array(x)
    else:
        x = np.arange(len(blknum_arr))
        bar_w = 0.3
        
    axs[0,0].bar(x,y_values[0],bar_w,color=colors[0])
    axs[0,0].set_title("Number of dependencies = 1", fontsize=12)
    axs[0,1].bar(x,y_values[1],bar_w,color=colors[1])
    axs[0,1].set_title("Number of dependencies = 2", fontsize=12)
    axs[1,0].bar(x,y_values[2],bar_w,color=colors[2])
    axs[1,0].set_title("Number of dependencies = 3", fontsize=12)
    axs[1,1].bar(x,y_values[3],bar_w,color=colors[3])
    axs[1,1].set_title("Number of dependencies = 4", fontsize=12)

    for ax in axs.flat:
        if isLog:
            ax.set_xscale('log')
        ax.set_xticks(x, blknum_arr, fontsize=12)
        ax.set_xlabel("Number of blocks", fontsize=12)
        ax.set_ylabel('Latency (s)', fontsize=12)
        ax.label_outer()
    
    if isLocal:
        axs[0,0].sharey(axs[0,1])
        yts = axs[1,1].get_yticks()
        yts = yts.astype(np.double)
        new_yts = np.arange(min(yts), max(yts), 0.1)
        axs[1,1].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].sharey(axs[1,1])
        axs[1,0].set_ylim(0,0.61)
        plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%g'))
    else:
        yts = axs[0,1].get_yticks()
        yts = yts.astype(np.double)
        new_yts = np.arange(min(yts), math.ceil(max(yts))+0.5, 0.5)
        axs[0,1].set_yticks(new_yts, new_yts, fontsize=12)
        axs[0,0].sharey(axs[0,1])
        axs[0,0].set_yticks(new_yts, new_yts, fontsize=12)

        yts = axs[1,1].get_yticks()
        yts = yts.astype(np.double)
        new_yts = np.arange(min(yts), max(yts), 0.5)
        print(new_yts)
        axs[1,1].set_yticks(new_yts, new_yts, fontsize=12)
        axs[1,0].set_yticks(new_yts, new_yts, fontsize=12)
        axs[1,0].sharey(axs[1,1])

    for ax in axs.flat:
        ax.grid(linestyle='--', color='darkgrey', axis='y', linewidth=0.4)

    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
v_kv_path = "./data/kv.csv"

# plot_kv_subplot(v_kv_path, "./plots/kv_sub.pdf", False, False)
# plot_kv_subplot(v_kv_path, "./plots/kv_sub_log.pdf", False, True)

In [ ]:
def plot_opc(input_path, output_path, isLog):
    w = 0.1
    colors = ['darksalmon','burlywood','darkseagreen','cornflowerblue']

    fig, ax = plt.subplots(1,1,figsize=set_size(width, fraction=1))

    input_data = pd.read_csv(input_path)
    y_values = list()
    
    if isLog:
        x = np.array([8,64,512,4096])
        for inp in inputnum_arr:
            y_values.append(input_data.loc[input_data['input_num'] == inp]['total'].tolist())
        for i in range(len(y_values)):
            ax.plot(x, y_values[i], marker="o", color=colors[i])
        plt.xscale('log')
        plt.yscale('log')
        plt.xticks(x, x, fontsize=12)
        plt.xlabel("Data size (KB)", fontsize=12)
        plt.legend(inputnum_arr.astype(str), fontsize=12, title="Number of dependencies", title_fontsize=12, ncol=2, bbox_to_anchor=(1,0.65))
      
        new_yts = [0.1,1,10,100]
        plt.yticks(new_yts, new_yts, fontsize=12)
    else:
        x = np.arange(len(datasz_arr))
        for ds in datasz_arr:
            y_values.append(input_data.loc[input_data['data_size'] == ds]['total'].tolist())
        for i in range(len(y_values)):
            ax.plot(x, y_values[i], marker="o", color=colors[i])

        plt.xticks(x, inputnum_arr, fontsize=12)
        plt.xlabel("Number of dependencies", fontsize=12)
        plt.legend(datasz_labels.astype(str), fontsize=12, title="Data size", title_fontsize=12, ncol=1, bbox_to_anchor=(1.25,0.75))
        
        yts = ax.get_yticks().astype(np.int64)
        plt.yticks(yts, yts, fontsize=12)
        plt.ylim([0, yts[-1]])

    
    
    print(y_values)
    
#     yts = ax.get_yticks().astype(np.int64)
#     plt.yticks(yts, yts, fontsize=12)
#     plt.ylim([0, yts[-1]])
    
    plt.ylabel("Latency (s)", fontsize=12)
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.4)
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
v_opc_path = "./data/opc.csv"

plot_opc(v_opc_path, "./plots/opc_log.pdf", True)
# plot_opc(v_opc_path, "./plots/opc.pdf", False)

In [ ]:
def plot_sign(input_path, output_path, isLog):
    w = 0.1
    colors = ['darksalmon','burlywood','darkseagreen','cornflowerblue']

    fig, ax = plt.subplots(1,1,figsize=set_size(width, fraction=1))

    input_data = pd.read_csv(input_path)
    x = np.arange(len(datasz_arr))
    y_values = list()

    if isLog:
        x = np.array([8,64,512,4096])
        for inp in inputnum_arr:
            y_values.append(input_data.loc[input_data['output_num'] == inp]['total'].tolist())
        for i in range(len(y_values)):
            ax.plot(x, y_values[i], marker="o", color=colors[i])
        plt.xscale('log')
        plt.yscale('log')
        plt.xticks(x, x, fontsize=12)
        plt.xlabel("Data size (KB)", fontsize=12)
        plt.legend(inputnum_arr.astype(str), fontsize=12, title="Number of outputs", title_fontsize=12, ncol=2, bbox_to_anchor=(1,0.65))
       
        new_yts = [0.1,1,10,100]
        plt.yticks(new_yts, new_yts, fontsize=12)
    else:
        x = np.arange(len(datasz_arr))
        for ds in datasz_arr:
            y_values.append(input_data.loc[input_data['data_size'] == ds]['total'].tolist())
        for i in range(len(y_values)):
            ax.plot(x, y_values[i], marker="o", color=colors[i])

        plt.xticks(x, inputnum_arr, fontsize=12)
        plt.xlabel("Number of outputs", fontsize=12)
        plt.legend(datasz_labels.astype(str), fontsize=12, title="Data size", title_fontsize=12, ncol=1, bbox_to_anchor=(1.25,0.75))
        yts = ax.get_yticks().astype(np.int64)
        plt.yticks(yts, yts, fontsize=12)
        plt.ylim([0, yts[-1]])

    
    print(y_values)

    
    plt.ylabel("Latency (s)", fontsize=12)
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.4)
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
sign_path = "./data/sign.csv"

plot_sign(sign_path, "./plots/sign_log.pdf", True)
# plot_sign(sign_path, "./plots/sign.pdf", False)

In [ ]:
def plot_dfu_mb(reg_path, dfu_path, output_path, isLog):
    fig, ax = plt.subplots()
    
    reg_data = pd.read_csv(reg_path)
    dfu_data = pd.read_csv(dfu_path)

    x = numctxt_arr
    reg_values = reg_data['total'].tolist()
    dfu_values = dfu_data['total'].tolist()
    
    print(reg_values)
    print(dfu_values)
    
    ax.plot(x, reg_values, marker="o", color='royalblue', label='Regular')
    ax.plot(x, dfu_values, marker="o", color='palevioletred', label='Protean')

    plt.xscale('log')
    plt.xticks(x, numctxt_arr, fontsize=12)

    if isLog:
        plt.yscale('log')
        if "shuf" in reg_path:
            new_yts = [1,10,100]
        else:
            new_yts = [0.1,1,10,100]
        plt.yticks(new_yts, new_yts, fontsize=12)
    else:
        yts = ax.get_yticks().astype(np.int64)
        if "shuf" in reg_path:
            new_yts = np.arange(0, max(yts)+1, 10)
        else:
            new_yts = np.arange(0, max(yts)+1, 5)
        plt.yticks(new_yts, new_yts, fontsize=12)
    
    plt.xlabel("Number of ciphertexts", fontsize=12)
    plt.ylabel("Latency (s)", fontsize=12)
    plt.legend(fontsize=12,ncol=1,bbox_to_anchor=(1.3,0.6))
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.4)
    
    if "shuf" in reg_path:
        plt.title("Shuffle protocol")
    else:
        plt.title("Threshold decryption protocol", fontsize=12)
    
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
# plot_dfu_mb('./data/shuf_reg.csv', './data/shuf_dfu.csv', './plots/shuf.pdf', False)
# plot_dfu_mb('./data/dec_reg.csv', './data/dec_dfu.csv', './plots/dec.pdf', False)

In [ ]:
plot_dfu_mb('./data/shuf_reg.csv', './data/shuf_dfu.csv', './plots/shuf_log.pdf', True)
plot_dfu_mb('./data/dec_reg.csv', './data/dec_dfu.csv', './plots/dec_log.pdf', True)

In [ ]:
def plot_txn(input_path, app_type, txn_type, step_size, isBatch):
    fig, ax = plt.subplots()

    input_data = pd.read_csv(input_path)
    x = numpart_arr
    y = input_data['total'].tolist()

    print(y)

    ax.plot(x, y, marker="o", color='mediumblue')

    glob_min = math.floor(min(y))
    glob_max = math.ceil(max(y))
    
    plt.xscale('log')
    plt.xticks(x, numpart_arr, fontsize=12)
    
    glob_min = 0
    new_yts = np.arange(glob_min, glob_max+step_size, step_size)
#     new_yts = np.arange(glob_min, glob_max+1, 1)
#     new_yts = np.arange(0,glob_max+5,5)
    plt.yticks(new_yts, new_yts, fontsize=12)

    plt.xlabel("Number of participants", fontsize=12)
    plt.ylabel('Latency (s)', fontsize=12)
        
    plt.title(f'{app_type} {txn_type} txn')
    plt.grid(linestyle='--', color='darkgrey', axis='y', linewidth=0.4)    
    if isBatch:
        output_path = os.path.join(os.getcwd(), "plots", f'{app_type.lower()}', f'{app_type.lower()}_{txn_type}_batch.pdf')
    else:
        output_path = os.path.join(os.getcwd(), "plots", f'{app_type.lower()}', f'{app_type.lower()}_{txn_type}.pdf')        
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
input_paths = ['./data/randlottery/close.csv', './data/randlottery/finalize.csv']

plot_txn(input_paths[0], 'RandLottery', 'close', 5, False)
plot_txn(input_paths[1], 'RandLottery', 'finalize', 5, False)

In [ ]:
input_paths = ['./data/randlottery/close_batch.csv', './data/randlottery/finalize_batch.csv']

plot_txn(input_paths[0], 'RandLottery', 'close', 5, True)
plot_txn(input_paths[1], 'RandLottery', 'finalize', 5, True)

In [ ]:
input_paths = ['./data/dkglottery/setup.csv', './data/dkglottery/close.csv', './data/dkglottery/finalize.csv']

plot_txn(input_paths[0], 'DKGLottery', 'setup', 5, False)
plot_txn(input_paths[1], 'DKGLottery', 'close', 5, False)
plot_txn(input_paths[2], 'DKGLottery', 'finalize', 10, False)

In [ ]:
input_paths = ['./data/dkglottery/setup_batch.csv', './data/dkglottery/close_batch.csv', './data/dkglottery/finalize_batch.csv']

plot_txn(input_paths[0], 'DKGLottery', 'setup', 5, True)
plot_txn(input_paths[1], 'DKGLottery', 'close', 5, True)
plot_txn(input_paths[2], 'DKGLottery', 'finalize', 10, True)

In [ ]:
input_paths = ['./data/evoting/setup.csv', './data/evoting/lock.csv', './data/evoting/shuffle.csv', './data/evoting/tally.csv']

plot_txn(input_paths[0], 'EVoting', 'setup', 5, False)
plot_txn(input_paths[1], 'EVoting', 'lock', 5, False)
plot_txn(input_paths[2], 'EVoting', 'shuffle', 10, False)
plot_txn(input_paths[3], 'EVoting', 'tally', 10, False)

In [ ]:
input_paths = ['./data/evoting/setup_batch.csv', './data/evoting/lock_batch.csv', './data/evoting/shuffle_batch.csv', './data/evoting/tally_batch.csv']

plot_txn(input_paths[0], 'EVoting', 'setup', 5, True)
plot_txn(input_paths[1], 'EVoting', 'lock', 5, True)
plot_txn(input_paths[2], 'EVoting', 'shuffle', 10, True)
plot_txn(input_paths[3], 'EVoting', 'tally', 10, True)

In [ ]:
def plot_jv_bar(input_path, output_path, app_type, txn_type, step_size, isBatch):
    fig, ax = plt.subplots()
    
    input_data = pd.read_csv(input_path)
    y_avgs = input_data['avg'].tolist()
    y_mins = input_data['min'].tolist()
    y_maxs = input_data['max'].tolist()
    y_stds = input_data['std'].tolist()

    xs = np.arange(len(numpart_arr))
    
    ax.bar(xs, y_avgs, width=0.3, color='sandybrown')
    ax.errorbar(xs, y_avgs, y_stds, solid_capstyle='projecting', fmt=' ', capsize=4, color='black')
#     ax.bar(xs, y_avgs, yerr=y_std, width=0.3, color='sandybrown')

    idx = 0
    for p in ax.patches:
        x = p.get_x()  # get the bottom left x corner of the bar
        w = p.get_width()  # get width of bar
        min_y = y_mins[idx]  # use h to get min from dict z
        max_y = y_maxs[idx]  # use h to get max from dict z
        plt.vlines(x+w/2, min_y, max_y, color='black')  # draw a vertical line
        idx+=1

    glob_max = math.ceil(max(y_maxs))
    new_yts = np.arange(0, glob_max+step_size, step_size)

    plt.xticks(xs, numpart_arr, fontsize=12)
    plt.yticks(new_yts, new_yts, fontsize=12)

    plt.xlabel("Number of participants", fontsize=12)
    plt.ylabel('Latency (s)', fontsize=12)
        
    plt.title(f'{app_type} {txn_type} txn')
    plt.grid(linestyle='--', color='darkgrey', axis='y', linewidth=0.4)
    if isBatch:
        output_path = os.path.join(os.getcwd(), "plots", f'{app_type.lower()}', f'{app_type.lower()}_{txn_type}_batch_bar.pdf')
    else:
        output_path = os.path.join(os.getcwd(), "plots", f'{app_type.lower()}', f'{app_type.lower()}_{txn_type}_bar.pdf')
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
# plot_jv_bar('./data/dkglottery/join.csv', '', 'DKGLottery', 'join', 5, False)
# plot_jv_bar('./data/randlottery/join.csv', '', 'RandLottery', 'join', 5, False)
# plot_jv_bar('./data/evoting/vote.csv', '', 'EVoting', 'vote', 5, False)

In [ ]:
# plot_jv_bar('./data/dkglottery/join_batch.csv', '', 'DKGLottery', 'join', 5, True)
# plot_jv_bar('./data/randlottery/join_batch.csv', '', 'RandLottery', 'join', 5, True)
# plot_jv_bar('./data/evoting/vote_batch.csv', '', 'EVoting', 'vote', 5, True)

In [ ]:
import matplotlib.patches as mpatches

def plot_jv_line(input_path, output_path, app_type, txn_type, step_size, isBatch):

    fig, ax = plt.subplots()
    
    input_data = pd.read_csv(input_path)
    y_avgs = input_data['avg'].tolist()
    y_mins = input_data['min'].tolist()
    y_maxs = input_data['max'].tolist()
    y_stds = input_data['std'].tolist()
    
    plt.xscale('log')
    xs = numpart_arr

    yerr0 = np.array(y_avgs) - np.array(y_stds)
    yerr1 = np.array(y_avgs) + np.array(y_stds)
    
    ax.plot(xs, y_avgs, marker='o', color='purple')
    ax.fill_between(xs, y_mins, y_maxs, alpha=0.4, color='thistle')
    ax.fill_between(xs, yerr0, yerr1, alpha=0.7, color='thistle')

    p1 = mpatches.Patch(facecolor='thistle', alpha=0.7, label='std')
    p2 = mpatches.Patch(facecolor='thistle', alpha=0.4, label='min/max')
    
    glob_max = math.ceil(max(y_maxs))
    new_yts = np.arange(0, glob_max+step_size, step_size)

    plt.xticks(xs, numpart_arr, fontsize=12)
    plt.yticks(new_yts, new_yts, fontsize=12)

    plt.xlabel("Number of participants", fontsize=12)
    plt.ylabel('Latency (s)', fontsize=12)
    
    plt.legend(handles=[p1,p2], fontsize=12, loc='upper left')
        
    plt.title(f'{app_type} {txn_type} txn')
    plt.grid(linestyle='--', color='darkgrey', axis='y', linewidth=0.4)
    if isBatch:
        output_path = os.path.join(os.getcwd(), "plots", f'{app_type.lower()}', f'{app_type.lower()}_{txn_type}_batch_line.pdf')
    else:
        output_path = os.path.join(os.getcwd(), "plots", f'{app_type.lower()}', f'{app_type.lower()}_{txn_type}_line.pdf')
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
plot_jv_line('./data/dkglottery/join.csv', '', 'DKGLottery', 'join', 5, False)
plot_jv_line('./data/randlottery/join.csv', '', 'RandLottery', 'join', 5, False)
plot_jv_line('./data/evoting/vote.csv', '', 'EVoting', 'vote', 5, False)

plot_jv_line('./data/dkglottery/join_batch.csv', '', 'DKGLottery', 'join', 5, True)
plot_jv_line('./data/randlottery/join_batch.csv', '', 'RandLottery', 'join', 5, True)
plot_jv_line('./data/evoting/vote_batch.csv', '', 'EVoting', 'vote', 5, True)